In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

from sklearn.datasets import make_regression

In [2]:
file = 'data_clean/sedan_limited_features.csv'
df = pd.read_csv(file,low_memory=False)

In [3]:
# fisker auto had a number of duplicate entries and was removed.
df = df.loc[df['make']!='FISKER AUTOMOTIVE']
df = df.reset_index(drop=True)

In [4]:
df.head()

,make,makeID,model,modelID,modelYear,age_in_years,mileage,askPrice,msrp,color
0,FORD,460.0,Taurus,1782.0,2015.0,5.0,67588,16462,16462,Ingot Silver Metallic
1,AUDI,582.0,S8,3678.0,2015.0,5.0,32990,59962,59962,Daytona Gray Pearl Effect
2,CHRYSLER,477.0,300,1878.0,2017.0,3.0,9231,23962,23962,Billet Silver Metallic Clearcoat
3,LINCOLN,464.0,MKZ,1790.0,2016.0,4.0,19710,22962,22962,White Platinum Metallic Tri-Coat
4,FORD,460.0,Fiesta,3267.0,2011.0,9.0,92053,5962,5962,Blue


In [5]:
df.count()

make            490456
makeID          490456
model           490456
modelID         490456
modelYear       490456
age_in_years    490456
mileage         490456
askPrice        490456
msrp            490456
color           490456
dtype: int64

In [ ]:
corr_set = df.corr(method='pearson')

corr_set

# Correlations

In [ ]:
import seaborn as sn
import matplotlib.pyplot as plt

fig = plt.figure(figsize = (19.20,10.80))

corrMatrix = df.corr()
sn.heatmap(corrMatrix, annot=True)
plt.savefig('visualizations/sedan_total_corr.png')

When observed as a whole, the corellations between certain features are below significancee

when each make is split into a group, the correlations are more easily observed

In [ ]:

fig = plt.figure(figsize = (19.20,10.80))

corrMatrix = df_honda.corr()
sn.heatmap(corrMatrix, annot=True)
plt.title('Honda')
plt.show()

In [ ]:
make_list = df['make'].unique()

for i, make in enumerate(make_list):
    df_new = df.loc[df['make']==f'{make}']
    df_new = df_new.drop(columns=['makeID'])
    fig = plt.figure(figsize = (19.20,10.80))

    corrMatrix = df_new.corr()
    sn.heatmap(corrMatrix, annot=True)
    plt.title(f'{make}')
    plt.savefig(f'visualizations/Sedan_Correlations/sedan_{make}_corr.png')
    plt.close()

with the overall data giving un-usable correlations we may need to have a regression model for each make/body class?

Test the difference between a prediction using the total data and a single make, data.

In [ ]:
df.head(1)

In [ ]:
df_test = df.drop(columns=['msrp','make','model',])

In [ ]:
df_test.head()

In [ ]:
X = df_test.drop(columns=['askPrice'])
y = df_test['askPrice']

In [ ]:
X_dummies = pd.get_dummies(X,drop_first=True)

In [ ]:
regr = RandomForestRegressor(max_depth=2, random_state=12)

In [ ]:
regr.fit(X_dummies, y)

In [ ]:
print(regr.predict(X_dummies))

In [ ]:
regr.score(X_dummies,y,sample_weight=None)

In [ ]:
df_honda = df.loc[df['make']=='HONDA']

In [ ]:
X = df_honda.drop(columns=['askPrice'])
y = df_honda['askPrice']

In [ ]:
X_dummies = pd.get_dummies(X,drop_first=True)

In [ ]:
regr = RandomForestRegressor(max_depth=2, random_state=12)

In [ ]:
regr.fit(X_dummies, y)

In [ ]:
print(regr.predict(X_dummies))

In [ ]:
regr.score(X_dummies,y,sample_weight=None)

In [6]:
FORD = df.loc[df['make']=='FORD']
AUDI = df.loc[df['make']=='AUDI']
CHRYSLER = df.loc[df['make']=='CHRYSLER']
LINCOLN = df.loc[df['make']=='LINCOLN']
INFINITI = df.loc[df['make']=='INFINITI']
BUICK = df.loc[df['make']=='BUICK']
NISSAN = df.loc[df['make']=='NISSAN']
CHEVROLET = df.loc[df['make']=='CHEVROLET']
BMW = df.loc[df['make']=='BMW']
TOYOTA = df.loc[df['make']=='TOYOTA']
VOLKSWAGEN = df.loc[df['make']=='VOLKSWAGEN']
HONDA = df.loc[df['make']=='HONDA']
MAZDA = df.loc[df['make']=='MAZDA']
VOLVO = df.loc[df['make']=='VOLVO']
JAGUAR = df.loc[df['make']=='JAGUAR']
DODGE = df.loc[df['make']=='DODGE']
CADILLAC = df.loc[df['make']=='CADILLAC']
MERCURY = df.loc[df['make']=='MERCURY']
HYUNDAI = df.loc[df['make']=='HYUNDAI']
KIA = df.loc[df['make']=='KIA']
MERCEDES_BENZ = df.loc[df['make']=='MERCEDES-BENZ']
SUBARU = df.loc[df['make']=='SUBARU']
ALFA_ROMEO = df.loc[df['make']=='ALFA ROMEO']
PONTIAC = df.loc[df['make']=='PONTIAC']
SAAB = df.loc[df['make']=='SAAB']
SATURN = df.loc[df['make']=='SATURN']
ACURA = df.loc[df['make']=='ACURA']
LEXUS = df.loc[df['make']=='LEXUS']
MITSUBISHI = df.loc[df['make']=='MITSUBISHI']
MASERATI = df.loc[df['make']=='MASERATI']
PORSCHE = df.loc[df['make']=='PORSCHE']
GENESIS = df.loc[df['make']=='GENESIS']
TESLA = df.loc[df['make']=='TESLA']
SUZUKI = df.loc[df['make']=='SUZUKI']
OLDSMOBILE = df.loc[df['make']=='OLDSMOBILE']
BENTLEY = df.loc[df['make']=='BENTLEY']
PLYMOUTH = df.loc[df['make']=='PLYMOUTH']
ROLLS_ROYCE = df.loc[df['make']=="ROLLS ROYCE"]
ASTON_MARTIN = df.loc[df['make']=='ASTON MARTIN']

In [7]:
df_list = [FORD, AUDI, CHRYSLER, LINCOLN, INFINITI, BUICK, NISSAN, CHEVROLET, BMW, TOYOTA, VOLKSWAGEN, HONDA, MAZDA, VOLVO, JAGUAR, DODGE, CADILLAC, MERCURY, HYUNDAI, KIA, MERCEDES_BENZ, SUBARU, ALFA_ROMEO, PONTIAC, SAAB, SATURN, ACURA, LEXUS, MITSUBISHI, MASERATI, PORSCHE, GENESIS, TESLA, SUZUKI, OLDSMOBILE, BENTLEY, PLYMOUTH, ROLLS_ROYCE, ASTON_MARTIN]
name_list = ['FORD', 'AUDI', 'CHRYSLER', 'LINCOLN', 'INFINITI', 'BUICK', 'NISSAN', 'CHEVROLET', 'BMW', 'TOYOTA', 'VOLKSWAGEN', 'HONDA', 'MAZDA', 'VOLVO', 'JAGUAR', 'DODGE', 'CADILLAC', 'MERCURY', 'HYUNDAI', 'KIA', 'MERCEDES_BENZ', 'SUBARU', 'ALFA_ROMEO', 'PONTIAC', 'SAAB', 'SATURN', 'ACURA', 'LEXUS', 'MITSUBISHI', 'MASERATI', 'PORSCHE', 'GENESIS', 'TESLA', 'SUZUKI', 'OLDSMOBILE', 'BENTLEY', 'PLYMOUTH', 'ROLLS_ROYCE', 'ASTON_MARTIN']

In [17]:
results_D4 = {}
score_D4 = {}
for i,df_element in enumerate(df_list):
    try:
        df_test = df_element.drop(columns=['msrp','make','model',])
        name = name_list[i]
        X = df_test.drop(columns=['askPrice'])
        y = df_test['askPrice']
        X_dummies = pd.get_dummies(X,drop_first=True)
        regr = RandomForestRegressor(max_depth=4, random_state=12)
        results_D4['model'] = regr
        regr.fit(X_dummies, y)
        results_D4[f'{name}_y-pred_D3'] = regr.predict(X_dummies)
        results_D4[f'{name}_score_D3']  = regr.score(X_dummies,y,sample_weight=None)
        score_D4[f'{name}_score_D3']  = regr.score(X_dummies,y,sample_weight=None)
        print(f'#{i} of {len(df_list)-1} - {name} complete...')
        print('----------------------------')
        print()
        
    except:
        print(f'{name} failed - moving on..')
        pass
print('done')

#0 of 38 - FORD complete...
----------------------------

#1 of 38 - AUDI complete...
----------------------------

#2 of 38 - CHRYSLER complete...
----------------------------

#3 of 38 - LINCOLN complete...
----------------------------

#4 of 38 - INFINITI complete...
----------------------------

#5 of 38 - BUICK complete...
----------------------------

#6 of 38 - NISSAN complete...
----------------------------

#7 of 38 - CHEVROLET complete...
----------------------------

#8 of 38 - BMW complete...
----------------------------

#9 of 38 - TOYOTA complete...
----------------------------

#10 of 38 - VOLKSWAGEN complete...
----------------------------

#11 of 38 - HONDA complete...
----------------------------

#12 of 38 - MAZDA complete...
----------------------------

#13 of 38 - VOLVO complete...
----------------------------

#14 of 38 - JAGUAR complete...
----------------------------

#15 of 38 - DODGE complete...
----------------------------

#16 of 38 - CADILLAC complete...
-

In [11]:
results

{'FORD_y-pred': array([15429.01006658,  8396.98691662, 15852.90912615, ...,
        15924.08474857, 15429.01006658, 15558.08303559]),
 'FORD_score': 0.01906870964752072,
 'AUDI_y-pred': array([29443.18832848, 31009.42827205, 38602.67130539, ...,
        38602.67130539, 31009.42827205, 18483.32381418]),
 'AUDI_score': 0.46073243258015206,
 'CHRYSLER_y-pred': array([21842.48573189, 13163.49315   , 14235.76359945, ...,
        14235.76359945, 14235.76359945, 21842.48573189]),
 'CHRYSLER_score': 0.7581386827041816,
 'LINCOLN_y-pred': array([22739.8153342 , 15174.59269781,  8086.62090273, ...,
        22739.8153342 , 22739.8153342 , 22739.8153342 ]),
 'LINCOLN_score': 0.7552783057735808,
 'INFINITI_y-pred': array([11163.07650753, 21284.33602563, 21598.46719831, ...,
        21284.33602563, 21598.46719831, 21598.46719831]),
 'INFINITI_score': 0.5598496300064364,
 'BUICK_y-pred': array([17485.14342744, 18527.86147186,  8141.67307673, ...,
        18527.86147186,  8141.67307673,  7353.25621558

In [31]:
pd.DataFrame(score, index=range(0,1)).T.reset_index()

,index,0
0,FORD_score,0.019069
1,AUDI_score,0.460732
2,CHRYSLER_score,0.758139
3,LINCOLN_score,0.755278
4,INFINITI_score,0.559850
5,BUICK_score,0.040571
6,NISSAN_score,0.001646
7,CHEVROLET_score,0.015137
8,BMW_score,0.456069
9,TOYOTA_score,0.012482


In [26]:
pd.DataFrame(score_D3, index=range(0,1)).T 

,0
FORD_score_D3,0.975099
AUDI_score_D3,0.569509
CHRYSLER_score_D3,0.831225
LINCOLN_score_D3,0.835990
INFINITI_score_D3,0.838106
BUICK_score_D3,0.665758
NISSAN_score_D3,0.801796
CHEVROLET_score_D3,0.083404
BMW_score_D3,0.522664
TOYOTA_score_D3,0.055156


In [25]:
pd.DataFrame(score_D4, index=range(0,1)).T

,0
FORD_score_D3,0.975099
AUDI_score_D3,0.656811
CHRYSLER_score_D3,0.868469
LINCOLN_score_D3,0.870590
INFINITI_score_D3,0.890743
BUICK_score_D3,0.754235
NISSAN_score_D3,0.801796
CHEVROLET_score_D3,0.165122
BMW_score_D3,0.579208
TOYOTA_score_D3,0.621093


In [13]:
make = 'NISSAN'

df.loc[df['make']==make]

,make,makeID,model,modelID,modelYear,age_in_years,mileage,askPrice,msrp,color
9,NISSAN,478.0,Altima,1904.0,2013.0,7.0,118091,8962,8962,Metallic Slate
51,NISSAN,478.0,Altima,1904.0,2017.0,3.0,47181,14462,14462,Super Black
54,NISSAN,478.0,Maxima,2302.0,2016.0,4.0,9593,25962,25962,Blue
64,NISSAN,478.0,Maxima,2302.0,2018.0,2.0,20495,22962,22962,Super Black
97,NISSAN,478.0,Altima,1904.0,2015.0,5.0,28446,13518,13718,Solid White
...,...,...,...,...,...,...,...,...,...,...
490411,NISSAN,478.0,Altima,1904.0,2019.0,1.0,29326,17799,18789,Super Black
490415,NISSAN,478.0,Maxima,2302.0,2017.0,3.0,45168,20675,21000,Pearl White
490425,NISSAN,478.0,Altima,1904.0,2016.0,4.0,106256,9910,10000,Gun Metallic
490426,NISSAN,478.0,Sentra,1907.0,2012.0,8.0,129414,5999,6000,Aspen White


In [14]:
df

,make,makeID,model,modelID,modelYear,age_in_years,mileage,askPrice,msrp,color
0,FORD,460.0,Taurus,1782.0,2015.0,5.0,67588,16462,16462,Ingot Silver Metallic
1,AUDI,582.0,S8,3678.0,2015.0,5.0,32990,59962,59962,Daytona Gray Pearl Effect
2,CHRYSLER,477.0,300,1878.0,2017.0,3.0,9231,23962,23962,Billet Silver Metallic Clearcoat
3,LINCOLN,464.0,MKZ,1790.0,2016.0,4.0,19710,22962,22962,White Platinum Metallic Tri-Coat
4,FORD,460.0,Fiesta,3267.0,2011.0,9.0,92053,5962,5962,Blue
...,...,...,...,...,...,...,...,...,...,...
490451,MERCEDES-BENZ,449.0,S-Class,2086.0,2015.0,5.0,32190,40999,48000,Designo Diamond White
490452,SUBARU,523.0,Legacy,2232.0,2018.0,2.0,76156,16990,19500,Crystal White Pearl
490453,AUDI,582.0,A6,3148.0,2016.0,4.0,58176,23850,27750,Florett Silver Metallic
490454,CHEVROLET,467.0,Cruze,1832.0,2017.0,3.0,30372,12950,14750,Tungsten Metallic


Run the random forest regression models
display the results to a dataframe for comparison
Row = Make score
Col = #_depth

use https://mljar.com/blog/save-load-random-forest/ to help with saving the models

turn regression finder into a repeating loop to moce through depth of trees and save dictionaries accordingly

alter layout to return lists instead of dictionaries?

need means of comfortablely making the data into a dataframe